### 0. Import libraries and define constants

In [20]:
import json
import time
import requests
import pandas as pd
from tqdm import tqdm


In [21]:
CREDENTIALS_FILE_PATH = "../spotify_key.config"
STORED_DATA_PATH = "../data/audio_features.json"

### 1. Retrieve and store audio features

In [22]:
class DataRetriever:
    def __init__(self):
        self.__auth_token = self.__get_auth_token()

    def __get_auth_token(self):
        with open(CREDENTIALS_FILE_PATH, "r") as input_file:
            credentials = input_file.read()
        client_id, client_secret = credentials.split("\n")

        response = requests.post(
            "https://accounts.spotify.com/api/token",
            {
                "grant_type": "client_credentials",
                "client_id": client_id,
                "client_secret": client_secret,
            },
        )

        auth_token = response.json()["access_token"]
        return auth_token.strip()

    def get_audio_features(self, track_ids):
        response = requests.get(
            f"https://api.spotify.com/v1/audio-features?ids={track_ids}",
            headers={"Authorization": f"Bearer {self.__auth_token}"},
        )

        audio_features = response.json()["audio_features"]

        return audio_features


In [23]:
class DataSaver:
    @staticmethod
    def store(audio_features):
        with open(STORED_DATA_PATH, "w") as output_file:
            json.dump(audio_features, output_file, indent=4)

### 2. Test the Spotify API

In [25]:
track_ids = list(set(pd.read_csv("../data/spotify_songs.csv")["track_id"].to_list()))

In [26]:
batch_size = 100
request_per_minute = 100
number_of_tracks = 10000
data_retriever = DataRetriever()

all_audio_features = []

for batch_index in tqdm(range(len(track_ids[:number_of_tracks]) // batch_size)):
    audio_features = data_retriever.get_audio_features(
        ",".join(track_ids[batch_index * batch_size : (batch_index + 1) * batch_size])
    )
    all_audio_features.extend(audio_features)

    time.sleep(60 / request_per_minute)

DataSaver.store(all_audio_features)


100%|██████████| 100/100 [01:18<00:00,  1.28it/s]
